In [1]:
# import libraries
import requests
import lxml.html as lh
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


#create a page to handle the website content
page = requests.get(url)


#store the contents of the website under doc
doc = lh.fromstring(page.content)


#parse the data that are stored between <tr>..<tr> of HTML
tr_elements = doc.xpath('//tr')

#creat empty list
col=[]
i = 0

# for each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    print(i,name)
    col.append((name,[]))


1 Postcode
2 Borough
3 Neighbourhood



In [2]:
#first row is the header,so data is stored on the second row onwards
for j in range(1, len(tr_elements)):
    T = tr_elements[j]
    
#if row is not of size 3, the //tr  data is not from our table    
    if len(T)!=3:
        break    
    
    # i is the index of our column    
    i=0
   
    #iterate through each element of the row     
    for t in T.iterchildren():
        data=t.text_content()
       
    # check if row is empty 
        if i>0:
    # convert any numerical value into integers          
                try:
                  data =int(data)
                except:
                  pass
   
    # append the data to the empty list of the i'th column   
        col[i][1].append(data)
        
    # increment i for the next column    
        i+=1

# check length of each column        
[len(C) for (title, C) in col]

[288, 288, 288]

In [3]:
#create the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [4]:
#move the last column to the first one
cols=df.columns.tolist()
cols

['Borough', 'Neighbourhood\n', 'Postcode']

In [5]:
cols = cols[-1:] + cols[:-1]
df=df[cols]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [6]:
#ignore rows with "Borough" == 'Not Assigned'
import numpy as np

df.replace("Not assigned", np.nan, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned\n
1,M2A,NaN,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [7]:
missing_data =df.isnull()
missing_data.head()

,Postcode,Borough,Neighbourhood
0,False,True,False
1,False,True,False
2,False,False,False
3,False,False,False
4,False,False,False


In [8]:
df.dropna(subset=["Borough"],axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n
5,M6A,North York,Lawrence Manor\n
6,M7A,Queen's Park,Not assigned\n
7,M9A,Etobicoke,Islington Avenue\n
8,M1B,Scarborough,Rouge\n
9,M1B,Scarborough,Malvern\n


In [9]:
#combine neighbourhoods with the same postcode
df_postcode=df.groupby(['Postcode','Borough'])['Neighbourhood\n'].apply(','.join).reset_index()
df_postcode.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge\n,Malvern\n"
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n"
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


In [10]:
#move \n from Neighbourhood column
df_postcode.rename(columns={'Neighbourhood\n': 'Neighbourhood'}, inplace=True)
df=df_postcode.replace('\n','', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
#if a cell has a 'Boroughbut' a 'Not assigned' , then the neighbourhood will be the same as the 'Borough'

df1=df[df['Neighbourhood'] == 'Not assigned']
df1

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [12]:
df.Neighbourhood[df.Neighbourhood =='Not assigned']=df.Borough
df1=df[df['Neighbourhood'] == "Queen's Park"]
df1

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [123]:
df.shape

(103, 3)